In [ ]:
pip install datasets

In [ ]:
pip install evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
model_dir = "D:\Ha Anh\Group16\Sourcecode\model"


<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Nguyen Ha Anh\AppData\Local\Temp\ipykernel_6196\3491708364.py:1: SyntaxWarning: invalid escape sequence '\H'
  model_dir = "D:\Ha Anh\Group16\Sourcecode\model"


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

C:\Users\Nguyen Ha Anh\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nguyen Ha Anh\.cache\huggingface\hub\models--VietAI--vit5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from datasets import Dataset
test_df = pd.read_json("test.json", orient="records", lines=True)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
import torch

def evaluate_model(model, tokenizer, dataset):
    predictions = []
    references = []

    for example in dataset:
        # Chuyển input_ids và attention_mask từ list sang tensor
        input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(model.device)
        attention_mask = torch.tensor(example["attention_mask"]).unsqueeze(0).to(model.device)

        # Chuyển nhãn (label) thành chuỗi tham chiếu
        reference_text = tokenizer.decode(example["labels"], skip_special_tokens=True)

        # Tạo dự đoán
        output = model.generate(input_ids=input_ids, attention_mask=attention_mask,
                                max_length=128, num_beams=5, early_stopping=True)
        prediction = tokenizer.decode(output[0], skip_special_tokens=True)

        predictions.append(prediction)
        references.append(reference_text)

    return predictions, references


In [ ]:
# Tính BLEU
def compute_bleu(predictions, references):
    from evaluate import load
    bleu_metric = load("sacrebleu")
    bleu_metric.add_batch(predictions=predictions, references=[[ref] for ref in references])
    return bleu_metric.compute()


In [ ]:
# Đánh giá mô hình
predictions, references = evaluate_model(model, tokenizer, test_dataset)
bleu_score = compute_bleu(predictions, references)
print("BLEU Score:", bleu_score["score"])


BLEU Score: 49.29084425519694


In [4]:
import re
def preprocess_informal(text, min_length=6):
    # Xử lý khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng dư thừa
    text = re.sub(r'\s+([?.!,])', r'\1', text)  # Xóa khoảng trắng dư trước dấu câu
    text = re.sub(r'([?.!])\1{2,}', r'\1', text)  # Giảm thiểu dấu câu lặp

    # Thay thế từ viết tắt tạm thời để không bị tách nhầm
    text = re.sub(r'\b(mr|dr|ms|mrs)\.', r'\1<ABBR>', text, flags=re.IGNORECASE)

    # Tách câu dựa trên dấu câu kết thúc
    sentences = re.split(r'(?<=\.|\?|\!)\s', text)

    # Phục hồi từ viết tắt
    sentences = [re.sub(r'<ABBR>', '.', s, flags=re.IGNORECASE) for s in sentences]

    # Chỉ giữ câu dài hơn min_length
    sentences = [s.strip() for s in sentences if len(s.strip()) >= min_length]

    return sentences


In [5]:
def informal_to_formal(text):
    # Tách câu và thêm <EOS> đã được xử lý trong preprocess_informal
    sentences = preprocess_informal(text)  # Output đã có <EOS> cho mỗi câu
    formal_sentences = []

    for sentence in sentences:
        # Chuyển câu sang định dạng tensor cho model
        inputs = tokenizer(
            sentence, return_tensors="pt", max_length=128, truncation=True
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Generate output từ model cho từng câu
        outputs = model.generate(**inputs, max_length=128, num_beams=5, early_stopping=True)

        # Chuyển output từ model về dạng văn bản và thêm vào danh sách formal_sentences
        formal_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
        formal_sentences.append(formal_sentence)

    # Nối các câu formal lại thành một văn bản hoàn chỉnh và trả về
    return ' '.join(formal_sentences)


In [6]:
model.eval()

example = "tầm này cả vstep và aptis đều hết lịch thì còn các nào để mở giới hạn ngoại ngữ trước hôm đăng ký học ko mn huhu"
formal_sentence = informal_to_formal(example)
print("Câu informal:", example)
print("Câu formal:", formal_sentence)

Câu informal: tầm này cả vstep và aptis đều hết lịch thì còn các nào để mở giới hạn ngoại ngữ trước hôm đăng ký học ko mn huhu
Câu formal: Trong khoảng thời gian này, nếu như cả Vstep và Aptis đều hết lịch, vậy có cách nào để mở giới hạn ngoại ngữ trước khi đăng ký học không ạ?


In [ ]:
model.eval()

example = "Ai đang cần tìm công việc online thì vô trang cá nhân tui có công việc nè lương ổn áp lắm tui cũng đang làm á"
formal_sentence = informal_to_formal(example)
print("Câu informal:", example)
print("Câu formal:", formal_sentence)

Câu informal: Ai đang cần tìm công việc online thì vô trang cá nhân tui có công việc nè lương ổn áp lắm tui cũng đang làm á
Câu formal: Nếu bạn đang cần tìm công việc online, xin vui lòng vào trang cá nhân tôi, tôi có công việc này, lương ổn định, tôi cũng đang làm.
